# 卷积神经网络(LeNet)

In [1]:
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
import zipfile
from tqdm import tqdm
from PIL import Image
from collections import namedtuple

from IPython import display
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.nn import init
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np


## 数据生成与读取

In [2]:
def load_data_fashion_mnist(batch_size, resize=None, root='data/FashionMNIST'):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)
    if sys.platform.startswith('win'):
        num_workers = 0  # 0表示不用额外的进程来加速读取数据
    else:
        num_workers = 4
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_iter, test_iter

batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

## 定义模型网络

In [3]:
class AlexNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4),     # (1, 224, 224)->(96, 55, 55)
            nn.ReLU(),               
            nn.MaxPool2d(3, 2),          # (96, 55, 55)->(96, 27, 27)
            nn.Conv2d(96, 256, 5, 1, 2), # (96, 27, 27)->(256, 27, 27)
            nn.ReLU(),
            nn.MaxPool2d(3, 2),          # (256, 27, 27)->(256, 13, 13)
            nn.Conv2d(256, 384, 3, 1, 1),# (256, 13, 13)->(384, 12, 12)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),# (384, 12, 12)->(384, 11, 11)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),# (384, 11, 11)->(256, 10, 10)
            nn.MaxPool2d(3, 2),
        )
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10),
        )
        
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0],-1))
        return output
    

In [4]:
loss = nn.CrossEntropyLoss()
net = AlexNet()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

In [5]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device
    acc_sum, n =0.0,0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(X.to(device)).argmax(dim=1)==y.to(device)).float().sum().cpu().item()
                net.train()
                
            else:
                if('is_training' in net.__code__.co_varnames):
                    acc_sum += (net(X, is_training = False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n   

## 训练

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
def train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0,0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))
            
lr, num_epochs = 0.001 , 10
train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on cuda
epoch 1, loss 0.8553, train acc 0.677, test acc 0.841, time 74.0 sec
epoch 2, loss 0.3676, train acc 0.864, test acc 0.872, time 71.1 sec
epoch 3, loss 0.3019, train acc 0.889, test acc 0.890, time 74.2 sec
epoch 4, loss 0.2647, train acc 0.902, test acc 0.894, time 73.6 sec
epoch 5, loss 0.2373, train acc 0.912, test acc 0.907, time 77.8 sec
epoch 6, loss 0.2186, train acc 0.918, test acc 0.907, time 69.0 sec
epoch 7, loss 0.2016, train acc 0.925, test acc 0.912, time 67.6 sec
epoch 8, loss 0.1826, train acc 0.932, test acc 0.914, time 67.7 sec
epoch 9, loss 0.1705, train acc 0.936, test acc 0.917, time 70.2 sec
epoch 10, loss 0.1560, train acc 0.941, test acc 0.914, time 70.8 sec
